---

# **Importing of Libraries**

- In this section, we have imported all the libraries that we will used to scrap yahoo finance historical data.

---

In [ ]:
import time
import datetime
import pandas as pd
from datetime import timezone
from datetime import datetime as dt
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [2]:
data = pd.read_csv('List of Stocks case study.csv')
print('Data Shape:', data.shape)
data.head()

Data Shape: (161, 2)


,Stock Name,SYMBOL
0,ACC LIMITED,ACC
1,ADANI PORT & SEZ LTD,ADANIPORTS
2,ADANI ENTERPRISES LIMITED,ADANIENT
3,ADANI POWER LTD,ADANIPOWER
4,AMBUJA CEMENTS LTD,AMBUJACEM


---

# **Yahoo Finance Scraper**

- In this section, we will scrap the items as highlighted in the red box in the following image.

![img](./images/yahoo-finance-webpage.PNG?raw=true)

- We will use the following code to extract records of last N days of the given ticker in CSV file.

---

In [5]:
%%time

# Initialize webdriver with options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome("chromedriver", options=chrome_options)
driver.get('https://finance.yahoo.com/')

for tiker in data['SYMBOL']:
    try:
        driver.find_element(by=By.XPATH, value='//*[@id="yfin-usr-qry"]').send_keys(Keys.CONTROL + "a")
        driver.find_element(by=By.XPATH, value='//*[@id="yfin-usr-qry"]').send_keys(tiker)
        driver.find_element(by=By.XPATH, value='//*[@id="yfin-usr-qry"]').send_keys(Keys.RETURN)
        time.sleep(3)
        # driver.find_element_by_xpath('//*[@id="quote-nav"]/ul/li[5]/a').click()
        driver.find_element(by=By.XPATH, value='//a[contains(@href, "history?p")]').click()

        time.sleep(3)
        ticker_name = driver.current_url.split('=')[1]

        # Estimate the timestamp of current day and the previous N days
        days = 30
        records = list()

        current_date = dt.now().date()
        current_date = dt.strptime(str(current_date), "%Y-%m-%d")
        current_timestamp = int(current_date.replace(tzinfo=timezone.utc).timestamp())

        previous_date = current_date - datetime.timedelta(days)
        previous_timestamp = int(previous_date.replace(tzinfo=timezone.utc).timestamp())
        
        time.sleep(3)
        driver.find_element(by=By.XPATH, value='//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/button').click()
        time.sleep(3)

        driver.get('https://finance.yahoo.com/quote/'+ ticker_name + '/history?period1=' + str(previous_timestamp) + '&period2=' + str(current_timestamp) + '&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true')

        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find('table', attrs={'data-test': 'historical-prices'}).find('tbody').find_all('tr')

        for result in results:
            row_list = list()
            row = result.findAll('span')
            for item in row:
                row_list.append(item.text)
            records.append(row_list)

        columns = ['Date', 'Open', 'High', 'Low', 'Close', 'AdjustedClose', 'Volume']

        data = pd.DataFrame(data=records, columns=columns)
        data.to_csv(path_or_buf='saves/'+ticker_name+'_stock_data.csv', index=False)
        print('Data for', ticker_name, 'scraped successfully!')
    except:
        print('Error occured while scraping data for', ticker_name)

Data for ACC scraped successfully!
Data for ADANIPORTS.NS scraped successfully!
Data for ADANIENT.NS scraped successfully!
Data for ADANIPOWER.NS scraped successfully!
Data for AMBUJACEM.NS scraped successfully!
Data for APOLLOHOSP.NS scraped successfully!
Data for ARVIND.NS scraped successfully!
Data for ASIANPAINT.NS scraped successfully!
Data for AUROPHARMA.NS scraped successfully!
Data for BAJFINANCE.NS scraped successfully!
Data for BALKRISIND.NS scraped successfully!
Data for BANKBARODA.NS scraped successfully!
Data for BANKINDIA.NS scraped successfully!
Data for BERGEPAINT.NS scraped successfully!
Data for BHEL.NS scraped successfully!
Data for BAJAJFINSV.NS scraped successfully!
Data for NVAX scraped successfully!
Data for NVAX scraped successfully!
Data for NVAX scraped successfully!
Data for CHOLAFIN.NS scraped successfully!
Data for NVAX scraped successfully!
Data for DIVISLAB.NS scraped successfully!
Data for NVAX scraped successfully!
Data for ENGINERSIN.NS scraped success